In [1]:
import pypsa 
import atlite 
import vresutils.shapes as vshapes
from vresutils.graph import voronoi_partition_pts
import geopandas as gpd
import powerplantmatching as pm
import pandas as pd
from scipy.spatial import KDTree
import numpy as np
import geopandas as gpd

from pypsa.opt import Constraint as Con

import matplotlib.pyplot as plt
%matplotlib inline
import xarray as xr

!hostname

resi


In [2]:
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
n = pypsa.Network('./vietnam3_storage.nc')

INFO:pypsa.io:Imported network vietnam3_storage.nc has buses, carriers, generators, lines, loads, storage_units


In [6]:
n.buses

,area,area_offshore,country,v_nom,x,y,substation_lv,type,carrier,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
name,,,,,,,,,,,,,,
HiepHoa,NaN,NaN,VN,500.0,105.904364,21.335112,True,,AC,1.0,0.0,inf,PQ,
LaiChauH,NaN,NaN,VN,500.0,102.984253,22.137909,True,,AC,1.0,0.0,inf,PQ,
DongAnh,NaN,NaN,VN,500.0,105.897505,21.163109,True,,AC,1.0,0.0,inf,PQ,
SonLa,NaN,NaN,VN,500.0,104.001312,21.537377,True,,AC,1.0,0.0,inf,PQ,
SonLaH,NaN,NaN,VN,500.0,103.996409,21.497521,True,,AC,1.0,0.0,inf,PQ,
MongDuong1,NaN,NaN,VN,500.0,107.346737,21.073995,True,,AC,1.0,0.0,inf,PQ,
MongDuong2,NaN,NaN,VN,500.0,107.349981,21.073477,True,,AC,1.0,0.0,inf,PQ,
QuangNinh,NaN,NaN,VN,500.0,107.130358,21.038549,True,,AC,1.0,0.0,inf,PQ,
NghiSon,NaN,NaN,VN,500.0,105.720356,19.315539,True,,AC,1.0,0.0,inf,PQ,


In [7]:
n.carriers

,marginal,capital,co2_emissions,short_name
name,,,,
bioenergy,72.900,256000,0.0,B
hard coal,30.000,222000,427.7,C
hydro,5.000,211200,0.0,H
ocgt,76.800,63800,230.0,G
nuclear,10.400,455100,0.0,N
oil,91.600,255300,405.0,O
wind,0.015,140400,0.0,W
solar,0.010,65920,0.0,S
perpetuum,1000.000,1000000,0.0,P


In [8]:
n.generators

,bus,carrier,component,p_nom,max_hours,p_nom_extendable,p_nom_min,p_nom_max,marginal_cost,capital_cost,weight,control,type,p_min_pu,p_max_pu,p_set,q_set,sign,efficiency,committable,start_up_cost,shut_down_cost,min_up_time,min_down_time,initial_status,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CongThanh hard coal,CongThanh,hard coal,PP,600.0,0.0,True,600.0,inf,0.030000,222.00,1.0,PQ,,0.0,1.0,0.0,0.0,1.0,1.0,False,0.0,0.0,0,0,1,NaN,NaN,1.0,1.0,0.0
DiLinh wind,DiLinh,wind,PP,30.0,0.0,True,30.0,1.022227e+05,0.000015,140.40,1.0,PQ,,0.0,1.0,0.0,0.0,1.0,1.0,False,0.0,0.0,0,0,1,NaN,NaN,1.0,1.0,0.0
DongAnh hard coal,DongAnh,hard coal,PP,440.0,0.0,True,440.0,inf,0.030000,222.00,1.0,PQ,,0.0,1.0,0.0,0.0,1.0,1.0,False,0.0,0.0,0,0,1,NaN,NaN,1.0,1.0,0.0
DuyenHai hard coal,DuyenHai,hard coal,PP,1866.0,0.0,True,1866.0,inf,0.030000,222.00,1.0,PQ,,0.0,1.0,0.0,0.0,1.0,1.0,False,0.0,0.0,0,0,1,NaN,NaN,1.0,1.0,0.0
HiepHoa hard coal,HiepHoa,hard coal,PP,235.0,0.0,True,235.0,inf,0.030000,222.00,1.0,PQ,,0.0,1.0,0.0,0.0,1.0,1.0,False,0.0,0.0,0,0,1,NaN,NaN,1.0,1.0,0.0
LongPhu wind,LongPhu,wind,PP,99.0,0.0,True,99.0,1.021848e+05,0.000015,140.40,1.0,PQ,,0.0,1.0,0.0,0.0,1.0,1.0,False,0.0,0.0,0,0,1,NaN,NaN,1.0,1.0,0.0
MongDuong2 hard coal,MongDuong2,hard coal,PP,2800.0,0.0,True,2800.0,inf,0.030000,222.00,1.0,PQ,,0.0,1.0,0.0,0.0,1.0,1.0,False,0.0,0.0,0,0,1,NaN,NaN,1.0,1.0,0.0
NhaBe ocgt,NhaBe,ocgt,PP,1215.0,0.0,True,1215.0,inf,0.076800,63.80,1.0,PQ,,0.0,1.0,0.0,0.0,1.0,1.0,False,0.0,0.0,0,0,1,NaN,NaN,1.0,1.0,0.0
NhaBe oil,NhaBe,oil,PP,654.0,0.0,True,654.0,inf,0.091600,255.30,1.0,PQ,,0.0,1.0,0.0,0.0,1.0,1.0,False,0.0,0.0,0,0,1,NaN,NaN,1.0,1.0,0.0


In [9]:
n.storage_units

,bus,carrier,component,p_nom,max_hours,p_nom_extendable,p_nom_min,p_nom_max,marginal_cost,capital_cost,efficiency_store,efficiency_dispatch,control,type,p_min_pu,p_max_pu,p_set,q_set,sign,state_of_charge_initial,state_of_charge_set,cyclic_state_of_charge,standing_loss,inflow,p_nom_opt
name,,,,,,,,,,,,,,,,,,,,,,,,,
ChonThanh hydro,ChonThanh,hydro,Store,282.00,1.037106,True,282.00,423.000000,0.005,211.2,0.90,0.90,PQ,,-1.0,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
DaNang hydro,DaNang,hydro,Store,21.00,5.537143,True,21.00,31.500000,0.005,211.2,0.90,0.90,PQ,,-1.0,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
DakNong hydro,DakNong,hydro,Store,1781.50,0.332788,True,1781.50,2672.250000,0.005,211.2,0.90,0.90,PQ,,-1.0,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
DiLinh hydro,DiLinh,hydro,Store,1141.10,0.257549,True,1141.10,1711.650000,0.005,211.2,0.90,0.90,PQ,,-1.0,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
DocSoi hydro,DocSoi,hydro,Store,166.10,0.572173,True,166.10,249.150000,0.005,211.2,0.90,0.90,PQ,,-1.0,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
HaTinh hydro,HaTinh,hydro,Store,55.30,1.445859,True,55.30,82.950000,0.005,211.2,0.90,0.90,PQ,,-1.0,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
HiepHoa hydro,HiepHoa,hydro,Store,30.39,11.760563,True,30.39,45.585000,0.005,211.2,0.90,0.90,PQ,,-1.0,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
HoaBinh hydro,HoaBinh,hydro,Store,615.10,1.462422,True,615.10,922.650000,0.005,211.2,0.90,0.90,PQ,,-1.0,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
LaiChau hydro,LaiChau,hydro,Store,464.10,0.124415,True,464.10,696.150000,0.005,211.2,0.90,0.90,PQ,,-1.0,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0


In [19]:
n.storage_units_t.inflow

name,CauBong hydro,ChonThanh hydro,CongThanh hydro,CuChi hydro,DaNang hydro,DakNong hydro,DiLinh hydro,DocSoi hydro,DongAnh hydro,DucHoa hydro,DuyenHai hydro,HaTinh hydro,HiepHoa hydro,HoaBinh hydro,LaiChau hydro,LaiChauH hydro,LongPhu hydro,LongThanh hydro,MongDuong1 hydro,MongDuong2 hydro,MyTho hydro,NamDinh1 hydro,NghiSon hydro,NhaBe hydro,NhoQuan hydro,OMon hydro,PhoNoi hydro,PhuLam hydro,PhuMy hydro,Pleiku hydro,Pleiku2 hydro,QuangNinh hydro,SonLa hydro,SonLaH hydro,SongMay hydro,TanDinh hydro,TanUyen hydro,TayHaNoi hydro,ThangLongT hydro,ThanhMy hydro,ThuongTin hydro,VanPhong hydro,VietTri hydro,VinhTan hydro,VungAng hydro,Yali hydro
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,0.000049,1.718738,0.001696,0.006133,2.707974,98.776680,155.571026,110.658989,0.000431,0.000249,0.000040,3.073706,0.592410,117.169320,36.578268,85.641225,0.006856,0.538410,0.012665,0.012518,0.003027,0.002065,21.630566,0.000528,8.850363,0.060798,0.010865,0.000060,0.007844,13.873866,16.442451,0.011278,166.045565,228.984420,1.017408,0.000329,0.000336,24.590270,0.095387,1508.996760,0.003302,96.035550,0.582232,0.360929,2.330152,119.911184
2015-01-01 01:00:00,0.000049,1.718736,0.001696,0.006123,2.717502,100.242906,155.759529,104.435563,0.000431,0.000249,0.000040,3.073655,0.592410,117.111728,36.645157,85.762269,0.006855,0.538410,0.012665,0.012518,0.003027,0.002065,21.630566,0.000528,8.850363,0.060697,0.010865,0.000060,0.007844,13.482027,16.000703,0.011278,166.562574,228.938557,1.017408,0.000329,0.000336,24.590270,0.095387,1480.890499,0.003302,99.033189,0.582232,0.371277,2.335609,110.474532
2015-01-01 02:00:00,0.000048,1.718736,0.001696,0.006119,2.694225,99.199293,152.444150,99.926353,0.000431,0.000249,0.000039,3.073217,0.591450,117.103567,36.682533,85.829318,0.006853,0.538333,0.012665,0.012518,0.003025,0.002068,21.630566,0.000528,8.849795,0.060647,0.010865,0.000060,0.007840,13.270300,15.880305,0.011278,166.961409,229.002124,1.015862,0.000329,0.000336,24.556058,0.095289,1437.854326,0.003301,92.714465,0.582188,0.358035,2.345092,104.375214
2015-01-01 03:00:00,0.000048,1.717980,0.001696,0.006117,2.674193,97.976122,149.691570,96.878628,0.000430,0.000249,0.000040,3.078832,0.590970,117.175249,36.712152,85.802321,0.006863,0.537711,0.012650,0.012476,0.003024,0.002069,21.628613,0.000528,8.849511,0.060678,0.010865,0.000060,0.007838,13.139953,15.663828,0.011278,166.979401,229.022653,1.015090,0.000329,0.000336,24.538953,0.095240,1366.456112,0.003301,86.878459,0.581695,0.347003,2.355288,100.292739
2015-01-01 04:00:00,0.000048,1.715366,0.001696,0.006116,2.658296,97.081978,147.589555,94.837701,0.000430,0.000249,0.000040,3.082868,0.590551,117.095063,36.711403,85.741975,0.006870,0.537338,0.012641,0.012451,0.003022,0.002070,21.627441,0.000527,8.849341,0.060679,0.010865,0.000060,0.007837,13.053971,15.520543,0.011278,167.079419,229.022034,1.014626,0.000329,0.000336,24.528689,0.095210,1312.657454,0.003301,82.119124,0.581400,0.339135,2.360531,97.725436
2015-01-01 05:00:00,0.000048,1.712735,0.001693,0.006110,2.644842,96.464653,146.208191,93.285539,0.000431,0.000249,0.000040,3.108997,0.590166,117.032396,36.649756,85.677607,0.006874,0.536730,0.012612,0.012441,0.003021,0.002071,21.606120,0.000527,8.842489,0.060653,0.010891,0.000060,0.007837,12.988432,15.433283,0.011243,166.759973,228.914632,1.013602,0.000328,0.000336,24.515137,0.095142,1273.721154,0.003301,78.896908,0.580894,0.333776,2.354695,95.932668
2015-01-01 06:00:00,0.000048,1.709068,0.001690,0.006103,2.632822,95.989472,145.150544,92.097564,0.000430,0.000249,0.000040,3.141324,0.589863,116.954123,36.558294,85.632035,0.006876,0.536281,0.012586,0.012422,0.003018,0.002071,21.594072,0.000527,8.842608,0.060594,0.010883,0.000060,0.007836,12.938330,15.352381,0.011220,166.445448,228.918817,1.013080,0.000329,0.000336,24.511182,0.095021,1244.634442,0.003301,76.563248,0.580603,0.329871,2.347130,94.576303
2015-01-01 07:00:00,0.000048,1.739412,0.001692,0.006198,2.586859,97.302923,145.689719,91.762538,0.000437,0.00

In [10]:
n.lines

,bus0,bus1,length,b,r,x,s_nom_extendable,capital_cost,type,g,s_nom,s_nom_min,s_nom_max,s_max_pu,terrain_factor,num_parallel,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
name,,,,,,,,,,,,,,,,,,,,,,,,,,
1,LaiChauH,SonLa,158.500,0.672611,4.390450,44.681150,True,0.063400,490-AL1/64-ST1A 380.0,0.0,0.0,0.0,inf,1.0,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LaiChau,SonLa,159.970,0.678849,4.431169,45.095543,True,0.063988,490-AL1/64-ST1A 380.0,0.0,0.0,0.0,inf,1.0,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LaiChau,LaiChauH,1.470,0.006238,0.040719,0.414393,True,0.000588,490-AL1/64-ST1A 380.0,0.0,0.0,0.0,inf,1.0,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SonLaH,SonLa,4.800,0.020369,0.132960,1.353120,True,0.001920,490-AL1/64-ST1A 380.0,0.0,0.0,0.0,inf,1.0,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,SonLa,HiepHoa,264.700,1.123281,7.332190,74.618930,True,0.105880,490-AL1/64-ST1A 380.0,0.0,0.0,0.0,inf,1.0,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,SonLa,VietTri,181.000,0.768092,5.013700,51.023900,True,0.072400,490-AL1/64-ST1A 380.0,0.0,0.0,0.0,inf,1.0,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,VietTri,HiepHoa,93.000,0.394655,2.576100,26.216700,True,0.037200,490-AL1/64-ST1A 380.0,0.0,0.0,0.0,inf,1.0,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,SonLa,LaiChauH,206.000,0.874182,5.706200,58.071400,True,0.082400,490-AL1/64-ST1A 380.0,0.0,0.0,0.0,inf,1.0,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,SonLaH,NhoQuan,206.000,0.874182,5.706200,58.071400,True,0.082400,490-AL1/64-ST1A 380.0,0.0,0.0,0.0,inf,1.0,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
